In [ ]:
states_abbreviations_mapper = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [93]:
import pandas as pd
import warnings
import plotly.graph_objects as go
import numpy as np

##I know I'm setting with a copy
warnings.filterwarnings("ignore",category=pd.core.common.SettingWithCopyWarning)

BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/'
TS_CONFIRMED_CASES = 'csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
TS_DEATH_CASES = 'csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
TS_RECOVERED_CASES = 'csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
mapbox_access_token = "pk.eyJ1IjoiandpbGxpczA3MjAiLCJhIjoiY2s4MHhoYmF6MDFoODNpcnVyNGR2bWk1bSJ9.YNwklD1Aa6DihVblHr3GVg"
#mapbox_style = "mapbox://styles/plotlymapbox/cjvprkf3t1kns1cqjxuxmwixz"
mapbox_style = 'dark'

def get_time_series(url):
    time_series_df = pd.read_csv(BASE_URL+url)
    return time_series_df.set_index(
        ['Province/State','Country/Region','Lat','Long']).transpose().unstack(
            fill_value="").reset_index().rename({'level_4':'Date',0:'Cases'},axis=1)

def parse_into_city(df):
    df['City'] = df['Province/State'].apply(
        lambda x: x.split(', ')[0] if len(x.split(', '))==2 else "")
    df['State'] = df['Province/State'].apply(
        lambda x: states_abbreviations_mapper[x.split(', ')[1].replace('.','').strip()] if len(x.split(', '))==2 else x)
    return df[df['City'] == ''],df[df['City'] != '']



# Confirmed Cases
confirmed_cases = get_time_series(TS_CONFIRMED_CASES)
us_confirmed_cases = confirmed_cases[confirmed_cases['Country/Region'] == 'US']
world_confirmed_cases = confirmed_cases[confirmed_cases['Country/Region'] != 'US']

# US Specific confirmed cases. Can parse out by city too
us_confirmed_cases_by_state, us_confirmed_cases_by_city = parse_into_city(
    us_confirmed_cases)

# Death Cases
death_cases = get_time_series(TS_DEATH_CASES)
us_death_cases = death_cases[death_cases['Country/Region'] == 'US']
world_death_cases = death_cases[death_cases['Country/Region'] != 'US']


# US Specific death cases. Can parse out by city too
us_death_cases_by_state, us_death_cases_by_city = parse_into_city(
    us_death_cases)

# Recovered Cases
recovered_cases = get_time_series(TS_RECOVERED_CASES)
us_recovered_cases = recovered_cases[recovered_cases['Country/Region'] == 'US']
world_recovered_cases = recovered_cases[recovered_cases['Country/Region'] != 'US']

# US Specific death cases. Can parse out by city too
us_recovered_cases_by_state, us_recovered_cases_by_city = parse_into_city(
    us_recovered_cases)

# Coerce dates
us_confirmed_cases_by_city['Date'] = pd.to_datetime(
    us_confirmed_cases_by_city['Date'], format='%m/%d/%y')

us_confirmed_cases_by_state['Date'] = pd.to_datetime(
    us_confirmed_cases_by_state['Date'], format='%m/%d/%y')

us_death_cases_by_city['Date'] = pd.to_datetime(
    us_death_cases_by_city['Date'], format='%m/%d/%y')

us_death_cases_by_state['Date'] = pd.to_datetime(
    us_death_cases_by_state['Date'], format='%m/%d/%y')

us_recovered_cases_by_state['Date'] = pd.to_datetime(
    us_recovered_cases_by_state['Date'], format='%m/%d/%y')

us_recovered_cases_by_state['Date'] = pd.to_datetime(
    us_recovered_cases_by_state['Date'], format='%m/%d/%y')

world_confirmed_cases['Date'] = pd.to_datetime(
    world_confirmed_cases['Date'], format='%m/%d/%y')

world_death_cases['Date'] = pd.to_datetime(
    world_death_cases['Date'], format='%m/%d/%y')

world_recovered_cases['Date'] = pd.to_datetime(
    world_recovered_cases['Date'], format='%m/%d/%y')



to_date_cases_by_city = us_confirmed_cases_by_city.groupby(
    ['City', 'State'], as_index=False).sum()


date_mapper = pd.DataFrame(
    us_confirmed_cases_by_city['Date'].unique(), columns=['Date'])
date_mapper['Date_text'] = date_mapper['Date'].dt.strftime('%m/%d/%y')

min_date = min(date_mapper.index)
max_date = max(date_mapper.index)


centroid_country_mapper = world_confirmed_cases.groupby('Country/Region').apply(lambda x: x.sort_values('Cases')[::-1].iloc[0][['Lat','Long']])
centroid_country_mapper = {x[0]: {'Long':x[1]['Long'],'Lat':x[1]['Lat']} for x in centroid_country_mapper.iterrows()}

def get_graph_state(date_int):
    print(date_int)
    # Get initial zoom and shit if the figure is already drawn
#     if not figure:

    lat = 36.017295
    lon = -39.471056
    zoom = 1.5
#     elif "layout" in figure:
#         lat = figure["layout"]["mapbox"]["lat"]
#         lon = figure["layout"]["mapbox"]["lon"]
#         zoom = figure["layout"]["mapbox"]["zoom"]

    official_date = date_mapper.iloc[date_int]['Date']
    print(date_int, official_date)

    df_confirmed = us_confirmed_cases_by_state[
        us_confirmed_cases_by_state['Date'] == official_date].groupby('Province/State Country/Region Lat Long'.split(),as_index=False).sum()
    df_confirmed.rename({'Cases':'Ongoing'},axis=1,inplace=1)
    df_confirmed['text'] = df_confirmed['Province/State'] + '<br>Total Cases {}: '.format(
        official_date.strftime('%m/%d/%y')) + df_confirmed['Ongoing'].astype(str)
    
    
    df_death = us_death_cases_by_state[
        us_death_cases_by_state['Date'] == official_date].groupby('Province/State Country/Region Lat Long'.split(),as_index=False).sum()
    df_death.rename({'Cases':'Deaths'},axis=1,inplace=1)
    df_death['text'] = df_death['Province/State'] + '<br>Total Deaths {}: '.format(
        official_date.strftime('%m/%d/%y')) + df_death['Deaths'].astype(str)
    
    
    df_recover = us_recovered_cases_by_state[
        us_recovered_cases_by_state['Date'] == official_date].groupby('Province/State Country/Region Lat Long'.split(),as_index=False).sum()
    df_recover.rename({'Cases':'Recovered'},axis=1,inplace=1)
    df_recover['text'] = df_recover['Province/State'] + '<br>Total Recovered {}: '.format(
        official_date.strftime('%m/%d/%y')) + df_recover['Recovered'].astype(str)
    


    
    sizeref = 2. * world_confirmed_cases.groupby(['Date','Country/Region']).sum().max()['Cases']/ (20 ** 2)

    
    # Has to take in a figure state eventually
    fig = go.Figure()
    fig.add_trace(go.Scattermapbox(
        lon=df_confirmed['Long']+np.random.normal(0,.02,len(df_confirmed['Long'])),
        lat=df_confirmed['Lat']+np.random.normal(0,.02,len(df_confirmed['Lat'])),
        text=df_confirmed['text'],
        customdata=df_confirmed['Province/State'],
        hoverinfo='text',
        textposition='top right',
        mode='markers',
        marker=dict(
            sizeref=sizeref,
            sizemin=3,
            size=df_confirmed['Ongoing'],
            color='yellow')))
    
    
    fig.add_trace(go.Scattermapbox(
        lon=df_death['Long']+np.random.normal(0,.02,len(df_death['Long'])),
        lat=df_death['Lat']+np.random.normal(0,.02,len(df_death['Long'])),
        text=df_death['text'],
        customdata=df_death['Province/State'],
        hoverinfo='text',
        textposition='top right',
        mode='markers',
        marker=dict(
            sizeref=sizeref,
            sizemin=2,
            size=df_death['Deaths'],
            color='red')))

    
    fig.add_trace(go.Scattermapbox(
        lon=df_recover['Long']+np.random.normal(0,.05,len(df_recover['Long'])),
        lat=df_recover['Lat']+np.random.normal(0,.05,len(df_recover['Lat'])),
        text=df_recover['text'],
        customdata=df_recover['Province/State'],
        hoverinfo='text',
        textposition='top right',
        mode='markers',
        marker=dict(
            sizeref=sizeref,
            sizemin=2,
            size=df_recover['Recovered'],
            color='green')))
    
    
    world_confirmed = world_confirmed_cases[
    world_confirmed_cases['Date'] == official_date].groupby('Country/Region'.split(),as_index=False).sum()
    world_confirmed.rename({'Cases':'Ongoing'},axis=1,inplace=1)
    world_confirmed['text'] = world_confirmed['Country/Region'] + '<br>Total Cases {}: '.format(
        official_date.strftime('%m/%d/%y')) + world_confirmed['Ongoing'].map('{:,}'.format)
    world_confirmed['Lat'] = world_confirmed['Country/Region'].apply(lambda x: centroid_country_mapper[x]['Lat'])
    world_confirmed['Long'] = world_confirmed['Country/Region'].apply(lambda x: centroid_country_mapper[x]['Long'])

    world_dead = world_death_cases[
        world_death_cases['Date'] == official_date].groupby('Country/Region'.split(),as_index=False).sum()
    world_dead.rename({'Cases':'Dead'},axis=1,inplace=1)
    world_dead['text'] = world_dead['Country/Region'] + '<br>Total Dead {}: '.format(
        official_date.strftime('%m/%d/%y')) + world_dead['Dead'].map('{:,}'.format)
    world_dead['Lat'] = world_dead['Country/Region'].apply(lambda x: centroid_country_mapper[x]['Lat'])
    world_dead['Long'] = world_dead['Country/Region'].apply(lambda x: centroid_country_mapper[x]['Long'])
    
    
    world_recovered = world_recovered_cases[
        world_recovered_cases['Date'] == official_date].groupby('Country/Region'.split(),as_index=False).sum()
    world_recovered.rename({'Cases':'Recovered'},axis=1,inplace=1)
    world_recovered['text'] = world_recovered['Country/Region'] + '<br>Total Recovered {}: '.format(
        official_date.strftime('%m/%d/%y')) + world_recovered['Recovered'].map('{:,}'.format)
    world_recovered['Lat'] = world_recovered['Country/Region'].apply(lambda x: centroid_country_mapper[x]['Lat'])
    world_recovered['Long'] = world_recovered['Country/Region'].apply(lambda x: centroid_country_mapper[x]['Long'])
    
    
    
    fig.add_trace(go.Scattermapbox(
        lon=world_confirmed['Long'],
        lat=world_confirmed['Lat'],
        text=world_confirmed['text'],
        customdata=world_confirmed['Country/Region'],
        hoverinfo='text',
        textposition='top right',
        mode='markers',
        marker=dict(
            sizeref=sizeref,
            sizemin=3,
            size=world_confirmed['Ongoing'],
            color='yellow')))
    
    fig.add_trace(go.Scattermapbox(
        lon=world_dead['Long']+np.random.normal(0,.05,len(world_dead['Long'])),
        lat=world_dead['Lat']+np.random.normal(0,.05,len(world_dead['Lat'])),
        text=world_dead['text'],
        customdata=world_dead['Country/Region'],
        hoverinfo='text',
        textposition='top right',
        mode='markers',
        marker=dict(
            sizeref=sizeref,
            sizemin=3,
            size=world_dead['Dead'],
            color='red')))
    
    fig.add_trace(go.Scattermapbox(
        lon=world_recovered['Long']+np.random.normal(0,.05,len(world_recovered['Long'])),
        lat=world_recovered['Lat']+np.random.normal(0,.05,len(world_recovered['Lat'])),
        text=world_recovered['text'],
        customdata=world_recovered['Country/Region'],
        hoverinfo='text',
        textposition='top right',
        mode='markers',
        marker=dict(
            sizeref=sizeref,
            sizemin=3,
            size=world_recovered['Recovered'],
            color='green')))


    layout = dict(
        title_text='The Corona is Coming',
        autosize=True,
        showlegend=False,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            style=mapbox_style,
            zoom=zoom,
            center=dict(lat=lat, lon=lon)
        ),
        hovermode="closest",
        margin=dict(r=0, l=0, t=0, b=0),
        dragmode="pan",
    )

    fig.update_layout(layout)
    return fig

fig = get_graph_state(date_mapper.index[-2])
fig['layout']

58
58 2020-03-20 00:00:00


Layout({
    'autosize': True,
    'dragmode': 'pan',
    'hovermode': 'closest',
    'mapbox': {'accesstoken': ('pk.eyJ1IjoiandpbGxpczA3MjAiLCJ' ... 'Wk1bSJ9.YNwklD1Aa6DihVblHr3GVg'),
               'center': {'lat': 36.017295, 'lon': -39.471056},
               'style': 'dark',
               'zoom': 1.5},
    'margin': {'b': 0, 'l': 0, 'r': 0, 't': 0},
    'showlegend': False,
    'template': '...',
    'title': {'text': 'The Corona is Coming'}
})

In [104]:

how_many_labels = (len(date_mapper))//10
marks = {k: {'label': v, 'style': {'color': 'grey', 'font-size': '1.5rem'}}
           for k, v in list(date_mapper['Date'].dt.strftime(
               '%m/%d/%y').to_dict().items())[::how_many_labels]}
last_key = list(marks.keys())[-1]
todays_key = date_mapper.index[-1]
if last_key == todays_key:
    marks[last_key]['label'] == 'Today'
else:
    marks[todays_key] = {'label': 'Today', 'style': {'color': 'grey', 'font-size': '1.5rem'}}

In [117]:
import time

In [121]:
official_date = date_mapper.iloc[0]['Date']
time.strftime("%Y-%m-%d", official_date)

TypeError: Tuple or struct_time argument required

In [127]:
official_date.strftime('%B %d, %Y')

'January 22, 2020'

In [29]:

world_confirmed = world_confirmed_cases[
world_confirmed_cases['Date'] == official_date].groupby('Country/Region'.split(),as_index=False).sum()
world_confirmed.rename({'Cases':'Ongoing'},axis=1,inplace=1)
world_confirmed['text'] = world_confirmed['Country/Region'] + '<br>Total Ongoing Cases {}: '.format(
official_date.strftime('%m/%d/%y')) + world_confirmed['Ongoing'].map('{:,}'.format)

centroid_lats = []
centroid_longs = []
for x in world_confirmed['Country/Region']:
    lat,long = world_confirmed_cases[world_confirmed_cases['Country/Region'] == x].sort_values('Cases')[::-1].iloc[0][['Lat','Long']]
    centroid_lats.append(lat)
    centroid_longs.append(long)

world_confirmed['Lat'] = centroid_lats
world_confirmed['Long'] = centroid_longs
world_confirmed[world_confirmed['Country/Region'] == 'China']

,Country/Region,Lat,Long,Ongoing,text
32,China,31.6927,88.0924,80977.0,"China<br>Total Ongoing Cases 03/14/20: 80,977.0"


[33.0,
 41.1533,
 28.0339,
 42.5063,
 -11.2027,
 17.0608,
 -38.4161,
 40.0691,
 -35.4735,
 47.5162,
 40.1431,
 25.0343,
 26.0275,
 23.685,
 13.1939,
 53.7098,
 50.8333,
 9.3077,
 27.5142,
 -16.2902,
 43.9159,
 -14.235,
 4.5353,
 42.7339,
 12.2383,
 16.5388,
 11.55,
 3.848,
 44.682,
 6.6111,
 15.4542,
 -35.6751,
 31.6927,
 4.5709,
 -4.0383,
 -4.0383,
 9.7489,
 7.54,
 45.1,
 35.4437,
 22.0,
 35.1264,
 49.8175,
 71.7069,
 11.8251,
 18.7357,
 -1.8312,
 26.0,
 13.7942,
 1.5,
 58.5953,
 -26.5225,
 9.145,
 -17.7134,
 64.0,
 -20.9043,
 -0.8037,
 13.4432,
 42.3154,
 51.0,
 7.9465,
 39.0742,
 15.7835,
 9.9456,
 5.0,
 18.9712,
 41.9029,
 15.2,
 47.1625,
 64.9631,
 21.0,
 -0.7893,
 32.0,
 33.0,
 53.1424,
 31.0,
 43.0,
 18.1096,
 36.0,
 31.24,
 48.0196,
 -0.0236,
 36.0,
 42.6026,
 29.5,
 41.2044,
 56.8796,
 33.8547,
 6.4281,
 47.14,
 55.1694,
 49.8153,
 -18.7669,
 2.5,
 3.2028,
 35.9375,
 14.6415,
 21.0079,
 -20.2,
 23.6345,
 47.4116,
 43.7333,
 46.8625,
 42.5,
 31.7917,
 -22.9576,
 28.1667,
 18.04

In [ ]:

world_confirmed = world_confirmed_cases[
    world_confirmed_cases['Date'] == official_date].groupby('Country/Region'.split(),as_index=False).aggregate(['sum','mean']).reset_index()
world_confirmed.rename({'Cases':'Ongoing'},axis=1,inplace=1)
world_confirmed['text'] = world_confirmed['Country/Region'] + '<br>Total Ongoing Cases {}: '.format(
    official_date.strftime('%m/%d/%y')) + world_confirmed['Ongoing']['sum'].map('{:,}'.format)

In [ ]:
world_confirmed = world_confirmed_cases[
    world_confirmed_cases['Date'] == official_date].groupby('Country/Region'.split(),as_index=False).sum()
world_confirmed.rename({'Cases':'Ongoing'},axis=1,inplace=1)
world_confirmed['text'] = world_confirmed['Country/Region'] + '<br>Total Ongoing Cases {}: '.format(
    official_date.strftime('%m/%d/%y')) + world_confirmed['Ongoing'].map('{:,}'.format)

centroid_lats = []
centroid_longs = []
for x in world_confirmed['Country/Region']:
    lat,long = world_confirmed[world_confirmed['Country/Region'] == x].sort_values('Ongoing')[::-1].iloc[0][['Lat','Long']]
    centroid_lats.append(lat)
    centroid_longs.append(long)

In [ ]:
aggreag = world_confirmed_cases[
    world_confirmed_cases['Date'] == official_date].groupby('Country/Region'.split(),as_index=False).max()

aggreag[aggreag['Country/Region'] == 'China']

In [ ]:
world_confirmed_cases[world_confirmed_cases['Country/Region'] == 'China'].groupby('Province/State').max()

In [ ]:
import pandas as pd
import warnings
import plotly.graph_objects as go
import numpy as np

##I know I'm setting with a copy
warnings.filterwarnings("ignore",category=pd.core.common.SettingWithCopyWarning)

BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/'
TS_CONFIRMED_CASES = 'csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
TS_DEATH_CASES = 'csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
TS_RECOVERED_CASES = 'csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
mapbox_access_token = "pk.eyJ1IjoiandpbGxpczA3MjAiLCJhIjoiY2s4MHhoYmF6MDFoODNpcnVyNGR2bWk1bSJ9.YNwklD1Aa6DihVblHr3GVg"
#mapbox_style = "mapbox://styles/plotlymapbox/cjvprkf3t1kns1cqjxuxmwixz"
mapbox_style = 'dark'

def get_time_series(url):
    time_series_df = pd.read_csv(BASE_URL+url)
    return time_series_df.set_index(
        ['Province/State','Country/Region','Lat','Long']).transpose().unstack(
            fill_value="").reset_index().rename({'level_4':'Date',0:'Cases'},axis=1)

def parse_into_city(df):
    df['City'] = df['Province/State'].apply(
        lambda x: x.split(', ')[0] if len(x.split(', '))==2 else "")
    df['State'] = df['Province/State'].apply(
        lambda x: states_abbreviations_mapper[x.split(', ')[1].replace('.','').strip()] if len(x.split(', '))==2 else x)
    return df[df['City'] == ''],df[df['City'] != '']



# Confirmed Cases
confirmed_cases = get_time_series(TS_CONFIRMED_CASES)
us_confirmed_cases = confirmed_cases[confirmed_cases['Country/Region'] == 'US']
world_confirmed_cases = confirmed_cases[confirmed_cases['Country/Region'] != 'US']

# US Specific confirmed cases. Can parse out by city too
us_confirmed_cases_by_state, us_confirmed_cases_by_city = parse_into_city(
    us_confirmed_cases)

# Death Cases
death_cases = get_time_series(TS_DEATH_CASES)
us_death_cases = death_cases[death_cases['Country/Region'] == 'US']
world_death_cases = death_cases[death_cases['Country/Region'] != 'US']


# US Specific death cases. Can parse out by city too
us_death_cases_by_state, us_death_cases_by_city = parse_into_city(
    us_death_cases)

# Recovered Cases
recovered_cases = get_time_series(TS_RECOVERED_CASES)
us_recovered_cases = recovered_cases[recovered_cases['Country/Region'] == 'US']
world_recovered_cases = recovered_cases[recovered_cases['Country/Region'] != 'US']

# US Specific death cases. Can parse out by city too
us_recovered_cases_by_state, us_recovered_cases_by_city = parse_into_city(
    us_recovered_cases)

# Coerce dates
us_confirmed_cases_by_city['Date'] = pd.to_datetime(
    us_confirmed_cases_by_city['Date'], format='%m/%d/%y')

us_confirmed_cases_by_state['Date'] = pd.to_datetime(
    us_confirmed_cases_by_state['Date'], format='%m/%d/%y')

us_death_cases_by_city['Date'] = pd.to_datetime(
    us_death_cases_by_city['Date'], format='%m/%d/%y')

us_death_cases_by_state['Date'] = pd.to_datetime(
    us_death_cases_by_state['Date'], format='%m/%d/%y')

us_recovered_cases_by_state['Date'] = pd.to_datetime(
    us_recovered_cases_by_state['Date'], format='%m/%d/%y')

us_recovered_cases_by_state['Date'] = pd.to_datetime(
    us_recovered_cases_by_state['Date'], format='%m/%d/%y')

world_confirmed_cases['Date'] = pd.to_datetime(
    world_confirmed_cases['Date'], format='%m/%d/%y')

world_death_cases['Date'] = pd.to_datetime(
    world_death_cases['Date'], format='%m/%d/%y')

world_recovered_cases['Date'] = pd.to_datetime(
    world_recovered_cases['Date'], format='%m/%d/%y')



to_date_cases_by_city = us_confirmed_cases_by_city.groupby(
    ['City', 'State'], as_index=False).sum()


date_mapper = pd.DataFrame(
    us_confirmed_cases_by_city['Date'].unique(), columns=['Date'])
date_mapper['Date_text'] = date_mapper['Date'].dt.strftime('%m/%d/%y')

min_date = min(date_mapper.index)
max_date = max(date_mapper.index)


def get_graph_state(date_int):
    # Get initial zoom and shit if the figure is already drawn
#     if not figure:

    lat = 36.017295
    lon = -39.471056
    zoom = 1.5
#     elif "layout" in figure:
#         lat = figure["layout"]["mapbox"]["lat"]
#         lon = figure["layout"]["mapbox"]["lon"]
#         zoom = figure["layout"]["mapbox"]["zoom"]

    official_date = date_mapper.iloc[date_int]['Date']
    print(date_int, official_date)

    df_confirmed = us_confirmed_cases_by_state[
        us_confirmed_cases_by_state['Date'] == official_date].groupby('Province/State Country/Region Lat Long'.split(),as_index=False).sum()
    df_confirmed.rename({'Cases':'Ongoing'},axis=1,inplace=1)
    df_confirmed['text'] = df_confirmed['Province/State'] + '<br>Total Ongoing Cases {}: '.format(
        official_date.strftime('%m/%d/%y')) + df_confirmed['Ongoing'].astype(str)
    
    
    df_death = us_death_cases_by_state[
        us_death_cases_by_state['Date'] == official_date].groupby('Province/State Country/Region Lat Long'.split(),as_index=False).sum()
    df_death.rename({'Cases':'Deaths'},axis=1,inplace=1)
    df_death['text'] = df_death['Province/State'] + '<br>Total Deaths {}: '.format(
        official_date.strftime('%m/%d/%y')) + df_death['Deaths'].astype(str)
    
    
    df_recover = us_recovered_cases_by_state[
        us_recovered_cases_by_state['Date'] == official_date].groupby('Province/State Country/Region Lat Long'.split(),as_index=False).sum()
    df_recover.rename({'Cases':'Recovered'},axis=1,inplace=1)
    df_recover['text'] = df_recover['Province/State'] + '<br>Total Recovered {}: '.format(
        official_date.strftime('%m/%d/%y')) + df_recover['Recovered'].astype(str)
    
    world_confirmed = world_confirmed_cases[
        world_confirmed_cases['Date'] == official_date].groupby('Country/Region',as_index=False).sum()
    world_confirmed.rename({'Cases':'Ongoing'},axis=1,inplace=1)
    world_confirmed['text'] = world_confirmed['Country/Region'] + '<br>Total Ongoing Cases {}: '.format(
        official_date.strftime('%m/%d/%y')) + world_confirmed['Ongoing'].map('{:,}'.format)
    
#     world_death = world_death_cases[
#         world_death_cases['Date'] == official_date].groupby('Country/Region Lat Long'.split(),as_index=False).sum()
#     world_death.rename({'Cases':'Ongoing'},axis=1,inplace=1)
#     world_death['text'] = world_death['Country/Region'] + '<br>Total Deaths {}: '.format(
#         official_date.strftime('%m/%d/%y')) + world_death['Ongoing'].map('{:,}'.format)
    
    
    sizeref = 2. * world_confirmed_cases.groupby(['Date','Country/Region']).sum().max()['Cases']/ (20 ** 2)

    
    # Has to take in a figure state eventually
    fig = go.Figure()
    fig.add_trace(go.Scattermapbox(
        lon=df_confirmed['Long']+np.random.normal(0,.02,len(df_confirmed['Long'])),
        lat=df_confirmed['Lat']+np.random.normal(0,.02,len(df_confirmed['Lat'])),
        text=df_confirmed['text'],
        customdata=df_confirmed['Province/State'],
        hoverinfo='text',
        textposition='top right',
        mode='markers',
        marker=dict(
            sizeref=sizeref,
            sizemin=3,
            size=df_confirmed['Ongoing'],
            color='yellow')))
    
    
    fig.add_trace(go.Scattermapbox(
        lon=df_death['Long']+np.random.normal(0,.02,len(df_death['Long'])),
        lat=df_death['Lat']+np.random.normal(0,.02,len(df_death['Long'])),
        text=df_death['text'],
        customdata=df_death['Province/State'],
        hoverinfo='text',
        textposition='top right',
        mode='markers',
        marker=dict(
            sizeref=sizeref,
            sizemin=2,
            size=df_death['Deaths'],
            color='red')))

    
    fig.add_trace(go.Scattermapbox(
        lon=df_recover['Long']+np.random.normal(0,.05,len(df_recover['Long'])),
        lat=df_recover['Lat']+np.random.normal(0,.05,len(df_recover['Lat'])),
        text=df_recover['text'],
        customdata=df_recover['Province/State'],
        hoverinfo='text',
        textposition='top right',
        mode='markers',
        marker=dict(
            sizeref=sizeref,
            sizemin=2,
            size=df_recover['Recovered'],
            color='green')))
    
    fig.add_trace(go.Scattermapbox(
        #lon=world_confirmed['Long']+np.random.normal(0,.02,len(world_confirmed['Long'])),
        #lat=world_confirmed['Lat']+np.random.normal(0,.02,len(world_confirmed['Lat'])),
        locations=world_confirmed['Country/Region'],
        locationmode='country name',
        text=world_confirmed['text'],
        customdata=world_confirmed['Country/Region'],
        hoverinfo='text',
        textposition='top right',
        mode='markers',
        marker=dict(
            sizeref=sizeref,
            sizemin=3,
            size=world_confirmed['Ongoing'],
            color='yellow')))
    
#     fig.add_trace(go.Scattermapbox(
#         lon=world_death['Long']+np.random.normal(0,.02,len(world_death['Long'])),
#         lat=world_death['Lat']+np.random.normal(0,.02,len(world_death['Lat'])),
#         text=world_death['text'],
#         customdata=world_death['Country/Region'],
#         hoverinfo='text',
#         textposition='top right',
#         mode='markers',
#         marker=dict(
#             sizeref=sizeref,
#             sizemin=3,
#             size=world_death['Ongoing'],
#             color='red')))


    layout = dict(
        title_text='The Corona is Coming',
        autosize=True,
        showlegend=False,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            style=mapbox_style,
            zoom=zoom,
            center=dict(lat=lat, lon=lon)
        ),
        hovermode="closest",
        margin=dict(r=0, l=0, t=0, b=0),
        dragmode="pan",
    )

    fig.update_layout(layout)
    return fig

fig = get_graph_state(date_mapper.index[-1])
fig

In [ ]:
centroid_country = centroids = pd.read_csv('country_centroids_az8.csv')
centroid = centroid_country[['admin','Longitude','Latitude']]


centroid_hash = {i[1]['admin']:{'Lat':i[1]['Latitude'],'Long':i[1]['Longitude']} for i in centroid.iterrows()}

In [ ]:
list(map(lambda x: centroid_hash[x]['Lat'], world_confirmed['Country/Region'].str.replace('Bahamas, The','The Bahamas')))

In [ ]:
centroid_hash.keys()